# 中断编程预置知识

> 什么是中断

一种硬件上的通知机制，用来通知CPU发生了某种需要立即处理的事件

分为：

1. 内部中断  
   
   CPU执行程序的过程中，发生的一些硬件出错、运算出错事件（如分母为0、溢出等等），不可屏蔽

2. 外部中断  
   
   外设发生某种情况，通过一个引脚的高、低电平变化来通知CPU （如外设产生了数据、某种处理完毕等等）

> 中断处理原理

任何一种中断产生，CPU都会暂停当前执行的程序，跳转到内存固定位置执行一段程序，该程序被称为总的中断服务程序，在该程序中区分中断源，然后进一步调用该中断源对应的处理函数。

中断源对应的处理函数被称为分中断处理程序，一般每一个分中断处理程序对应一个外设产生的中断

写驱动时，如果外设有中断，则需要编写一个函数（分中断处理程序）来处理这种中断

> 如何实现中断

在ARM裸机代码中需要配置：

1. I/O口为中断模式，触发方式，I/O口中断使能
2. 设置GIC中断使能，分发配置，分发总使能，CPU外部中断接口使能，中断优先级

在linux内核中，只需要：

1. 中断号是什么，怎么得到中断号
2. 中断处理方法

> 什么是中断号？

中断号就是一个号码，需要通过一定的方式获取，在linux3.14内核中，从设备树中获取

![alt text](中断号.png)

> 获取中断号的方法：

1. 宏定义
   
IRQ_EINT(号码)

2. 设备树文件中
   
arch/arm/boot/dts/exynos4412-fs4412.dts


设备树文件：arch/arm/boot/dts/exynos4x12-pinctrl.dtsi

```kernel
gpx1: gpx1 {
      gpio-controller;
      #gpio-cells = <2>;

      interrupt-controller;
      interrupt-parent = <&gic>;
      interrupts = <0 24 0>, <0 25 0>, <0 26 0>, <0 27 0>,
                  <0 28 0>, <0 29 0>, <0 30 0>, <0 31 0>;
      #interrupt-cells = <2>;
   };
```

> 硬件连接：

查看原理图，找到按键所对应的中断号SPI Port No
		
![alt text](./按键的硬件连接.png)

![alt text](./按键的硬件连接2.png)

![alt text](./中断表.png)

![alt text](./设备树的中断号信息.png)

# 中断编程的准备操作

在编程过程中，需要定义自己的节点–描述当前设备用的中断号

 arch/arm/boot/dts/exynos4412-fs4412.dts 

```kernel
    key_int_node{
        compatible = "test_key";
        interrupt-parent = <&gpx1>;
        interrupts = <2 4>;  	//2表示第几个中断号，4表示触发方式为下降沿
    };
```

interrupts = <2 4>;  

中断号2的定位方法有两种：

1. 可以在设备树文件arch/arm/boot/dts/exynos4x12-pinctrl.dtsi数是第几个

2. I/O引脚是gpx1_2,中断号就是2；I/O引脚是gpx1_7,中断号就是7

编译设备树文件：

	make dtbs

更新dtbs文件：

	cp -raf arch/arm/boot/dts/exynos4412-fs4412.dtb  /tftpboot/

开发板上电重启后，可以在/proc/device-tree/目录中看到我们添加的按键中断节点key_int_node

```kernel
mykey2_node {
	compatible = "mykey2,key2";
	key2-gpio = <&gpx1 1 0>;
	interrupt-parent = <&gpx1>;
	interrupts = <1 3>;
};

mykey3_node {
	compatible = "mykey3,key3";
	key2-gpio = <&gpx1 2 0>;
	interrupt-parent = <&gpx1>;
	interrupts = <2 3>;
};
```

以下是代码的详细解释：

- `compatible = "mykey2,key2";`：此行用于将设备驱动程序与设备匹配。字符串"mykey2"和"key2"用于标识兼容的设备和驱动程序。

- `key2-gpio = <&gpx1 1 0>;`：此行为键定义了一个GPIO（通用输入/输出）属性。GPIO来自gpx1控制器，在引脚1处，且为低电平有效（0）。

- `interrupt-parent = <&gpx1>;`：此行指定了此设备的中断控制器是gpx1。

- `interrupts = <1 3>;`：此行定义了设备的中断号和类型。中断号是1，类型是3（下降沿）。

请注意，这些值的确切含义可能会根据您使用的具体硬件和Linux内核版本而有所不同。

# 中断API

> 中断申请

```c
int request_irq(unsigned int irq, irq_handler_t handler, unsigned long flags,const char *name, void *dev)
```
参数：
1. irq：所申请的中断号

2. handler：该中断号对应的中断处理函数
	
3. flags：中断触发方式或处理方式 
		
    触发方式：
        
    + IRQF_TRIGGER_NONE 		//无触发
    + IRQF_TRIGGER_RISING 	//上升沿触发
    + IRQF_TRIGGER_FALLING  //下降沿触发
    + IRQF_TRIGGER_HIGH  	//高电平触发
    + IRQF_TRIGGER_LOW 		//低电平触发
		
    处理方式：
			   
    + IRQF_DISABLED		//用于快速中断，处理中屏蔽所有中断
	+ IRQF_SHARED		  //共享中断

4. name：中断名 /proc/interrupts
		
5. dev：传递给中断例程的参数，共享中断时用于区分那个设备，一般为对应设备的结构体地址，无共享中断时写NULL
    
返回值：成功：0 失败：错误码

> 中断释放

```c
void free_irq(unsigned int irq, void *dev_id)；
```

功能：释放中断号

参数：

irq：设备号

dev_id：共享中断时用于区分那个设备一般强转成设备号，无共享中断时写NULL

> 中断处理函数原型

```c
typedef irqreturn_t (*irq_handler_t)(int, void *);
```

参数：

int：中断号

void*:对应的申请中断时的dev_id

返回值：
```c
	typedef enum irqreturn irqreturn_t;	//中断返回值类型

	enum irqreturn {
		IRQ_NONE	= (0 << 0),
		IRQ_HANDLED	= (1 << 0),
		IRQ_WAKE_THREAD	= (1 << 1),
	};
```
返回IRQ_HANDLED表示处理完了,返回IRQ_NONE在共享中断表示不处理
